In [ ]:
# ---- UTILIZANDO a BASE MUSIC_STORE

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/music-store.db')
def consulta(query):
    return pd.read_sql(query, conn)

In [125]:
# Qual artista possui mais albuns? 

consulta("""
         SELECT
            a.name ActorName,
         COUNT(ab.id) as QtyAlbums
         FROM artists a
            INNER JOIN albums ab 
            ON a.id = ab.artist_id 
         GROUP BY artist_id
         ORDER BY QtyAlbums DESC
         LIMIT 1
         """)

,ActorName,QtyAlbums
0,Iron Maiden,21


In [31]:
# Existem clientes de quantos países? 

consulta("SELECT COUNT (DISTINCT country) AS Qty_Contries FROM CUSTOMERS")

,Qty_Contries
0,24


In [17]:
# Consultar quantidades de clientes por países

consulta("""
         SELECT 
            country,
         COUNT(id) as QtyClient 
         FROM customers c
         GROUP BY country
         ORDER BY QtyClient DESC
         """)


,country,QtyClient
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4
5,United Kingdom,3
6,Portugal,2
7,India,2
8,Czech Republic,2
9,Sweden,1


In [126]:
consulta("""
         SELECT
            country, 
         COUNT(id) as QtyClient
         FROM customers c
         WHERE country='Brazil'
         GROUP BY country 
         ORDER BY QtyClient DESC
         """)

,country,QtyClient
0,Brazil,5


In [165]:
# Qual genero possui mais músicas?

consulta("""
         SELECT
            g.name genres,
         COUNT(genre_id) as QtyGenres
         FROM tracks t
            INNER JOIN genres g 
            ON t.genre_id = g.id
         GROUP BY genres
         ORDER BY QtyGenres DESC
         LIMIT 1
         """)




,genres,QtyGenres
0,Rock,1297


In [19]:
# Qual genero possui mais albuns? 

consulta("""
         SELECT
            g.name genres,
         COUNT(DISTINCT album_id) as QtyAlbums
         FROM tracks t
            INNER JOIN genres g 
            ON t.genre_id = g.id
         GROUP BY genres
         ORDER BY QtyAlbums DESC
         LIMIT 1    
         """)

,genres,QtyAlbums
0,Rock,117


In [163]:
consulta("""
         SELECT
         COUNT(DISTINCT album_id) as QtyAlbums
         FROM tracks t
            INNER JOIN genres g 
            ON t.genre_id = g.id
        WHERE g.name = 'Rock'       
         """)   

,QtyAlbums
0,117


In [18]:
# Existem albuns que possuem mais de um gênero 

consulta("""
            SELECT 
                album_id, COUNT(DISTINCT genre_id) AS QtyGenres
            FROM Tracks
            GROUP BY album_id
            ORDER BY QtyGenres DESC 
""")

,album_id,QtyGenres
0,141,3
1,227,3
2,73,2
3,102,2
4,109,2
...,...,...
342,343,1
343,344,1
344,345,1
345,346,1


In [95]:
# Exiba os 5 clientes que mais fizeram pedidos. Qual é o valor da maior compra?

consulta("""
         SELECT
            customer_id,
            first_name ||' '|| last_name as fullname,
        COUNT(i.id) as QtyOrder,
        SUM(total) as TotalOrder
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY fullname
        ORDER BY QtyOrder DESC, TotalOrder DESC
        LIMIT 5   
         """)

,customer_id,fullname,QtyOrder,TotalOrder
0,6,Helena Holý,7,49.62
1,26,Richard Cunningham,7,47.62
2,57,Luis Rojas,7,46.62
3,46,Hugh O'Reilly,7,45.62
4,45,Ladislav Kovács,7,45.62


In [3]:
# Quais os emails dos 10 clientes que mais gastaram na loja? 

consulta("""
         SELECT
            customer_id,
            first_name ||' '|| last_name as fullname,
            email,
        SUM(total) as TotalPrice
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY fullname
        ORDER BY TotalPrice DESC
        LIMIT 10   
         """)

,customer_id,fullname,email,TotalPrice
0,6,Helena Holý,hholy@gmail.com,49.62
1,26,Richard Cunningham,ricunningham@hotmail.com,47.62
2,57,Luis Rojas,luisrojas@yahoo.cl,46.62
3,45,Ladislav Kovács,ladislav_kovacs@apple.hu,45.62
4,46,Hugh O'Reilly,hughoreilly@apple.ie,45.62
5,28,Julia Barnett,jubarnett@gmail.com,43.62
6,37,Fynn Zimmermann,fzimmermann@yahoo.de,43.62
7,24,Frank Ralston,fralston@gmail.com,43.62
8,25,Victor Stevens,vstevens@yahoo.com,42.62
9,7,Astrid Gruber,astrid.gruber@apple.at,42.62


In [14]:
# Qual o valor médio de cada compra? 

consulta('SELECT AVG(total) AS avg_price FROM invoices')
    

,avg_price
0,5.651942


In [6]:
# Qual o valor médio de cada compra? 

consulta("""
         SELECT
            customer_id,
            first_name ||' '|| last_name as fullname,
        COUNT(i.id) as QtyOrders,
        AVG(i.total) AVGOrders
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY fullname
        ORDER BY AVGOrders DESC
        LIMIT 10   
         """)

,customer_id,fullname,QtyOrders,AVGOrders
0,6,Helena Holý,7,7.088571
1,26,Richard Cunningham,7,6.802857
2,57,Luis Rojas,7,6.660000
3,45,Ladislav Kovács,7,6.517143
4,46,Hugh O'Reilly,7,6.517143
5,28,Julia Barnett,7,6.231429
6,37,Fynn Zimmermann,7,6.231429
7,24,Frank Ralston,7,6.231429
8,59,Puja Srivastava,6,6.106667
9,25,Victor Stevens,7,6.088571


In [30]:
# Quantas compras estão acima da média?

consulta("""
        SELECT 
        COUNT(total) AS avg_price,
        CASE
            WHEN total > (SELECT AVG(total) AS avg_price FROM invoices) THEN 'AcimaAVG'
            WHEN total < (SELECT AVG(total) AS avg_price FROM invoices) THEN 'abaixoAVG'
        END as categoria
        FROM invoices
        GROUP BY categoria
        """)




,avg_price,categoria
0,179,AcimaAVG
1,233,abaixoAVG


In [22]:
# Vamos fazer uma playlist com as 12 músicas mais vendidas! 
# Quais foram elas? Quais os generos e os artistas?

consulta("""SELECT
                t.id trackID,
                t.name MusicName,
                at.name Artist,
                g.name Genre,
            COUNT(track_id) AS QtyTrackSold
            FROM invoice_lines il
                INNER JOIN tracks t
                ON il.track_id = t.id
                INNER JOIN genres g
                ON g.id = t.genre_id
                INNER JOIN albums a
                ON a.id = t.album_id
                INNER JOIN artists at
                ON at.id = a.artist_id
             GROUP BY trackID
             ORDER BY QtyTrackSold DESC
             LIMIT 12     
            """)

,trackID,MusicName,Artist,Genre,QtyTrackSold
0,2,Balls to the Wall,Accept,Rock,2
1,8,Inject The Venom,AC/DC,Rock,2
2,9,Snowballed,AC/DC,Rock,2
3,20,Overdose,AC/DC,Rock,2
4,32,Deuces Are Wild,Aerosmith,Rock,2
5,48,Not The Doctor,Alanis Morissette,Rock,2
6,66,Por Causa De Você,Antônio Carlos Jobim,Jazz,2
7,84,Welcome Home (Sanitarium),Apocalyptica,Metal,2
8,161,Snowblind,Black Sabbath,Metal,2
9,162,Cornucopia,Black Sabbath,Metal,2


In [ ]:
# DESAFIO DO PROF: Precisamos definir 2 países para ações de marketing. 
# Para um, iremos direcionar campanhas dos albums mais caros, 
# nosso foco são os clientes que não se preocupam com o valor do pedido. 
# Para o outro grupo, que gasta menos, precisamos ser mais assertivos, 
# ecomendando albums mais baratos mas com músicas populares. 
# Qual seria sua estratégia? Quais países poderíamos atender? 
# Quais albuns iremos recomendar para cada um dos países? 
# Apresente a query (ou as queries), a tabela (ou as tabelas) 
# e justifique suas escolhas.

In [20]:
# Lista dos 20 países que possuem o maior valor per capita de vendas por pessoa

consulta("""
         SELECT
            c.country as Nationality,
        COUNT(DISTINCT c.id) as QtyCustomer,
        COUNT(i.id) as QtyInvoices,
        SUM(total) as TotalAmount,
        SUM(total)/COUNT(DISTINCT c.id) as PC,
        SUM(total)/COUNT(i.id) as PC2
        FROM invoices i
            INNER JOIN customers c 
            ON i.customer_id = c.id 
        GROUP BY Nationality
        ORDER BY PC DESC, PC2 DESC, TotalAmount DESC
        LIMIT 20   
         """)

,Nationality,QtyCustomer,QtyInvoices,TotalAmount,PC,PC2
0,Chile,1,7,46.62,46.620000,6.660000
1,Hungary,1,7,45.62,45.620000,6.517143
2,Ireland,1,7,45.62,45.620000,6.517143
3,Czech Republic,2,14,90.24,45.120000,6.445714
4,Austria,1,7,42.62,42.620000,6.088571
5,Finland,1,7,41.62,41.620000,5.945714
6,Netherlands,1,7,40.62,40.620000,5.802857
7,USA,13,91,523.06,40.235385,5.747912
8,Norway,1,7,39.62,39.620000,5.660000
9,Germany,4,28,156.48,39.120000,5.588571


In [3]:
# Lista dos 10 albuns mais caros

consulta ("""
            SELECT
                a.title AlbumName,
                at.name ArtistName,
                a.id AlbumID,
                SUM(unit_price) AlbumPrice,
                COUNT(t.id) Hits   
            FROM tracks t
            JOIN albums a
                ON t.album_id = a.id
            JOIN artists at
                ON a.artist_id = at.id
            GROUP BY AlbumName
            ORDER BY AlbumPrice DESC
        """)

,AlbumName,ArtistName,AlbumID,AlbumPrice,Hits
0,Greatest Hits,Lenny Kravitz,141,56.43,57
1,"Lost, Season 3",Lost,229,51.74,26
2,"The Office, Season 3",The Office,251,49.75,25
3,"Lost, Season 1",Lost,230,49.75,25
4,"Lost, Season 2",Lost,231,47.76,24
...,...,...,...,...,...
342,Allegri: Miserere,"Richard Marlow & The Choir of Trinity College,...",273,0.99,1
343,Adorate Deum: Gregorian Chant from the Proper ...,Alberto Turco & Nova Schola Gregoriana,272,0.99,1
344,"Adams, John: The Chairman Dances",Edo de Waart & San Francisco Symphony,307,0.99,1
345,A Soprano Inspired,"Britten Sinfonia, Ivor Bolton & Lesley Garrett",285,0.99,1
